<a href="https://colab.research.google.com/github/lgy112112/FUNet/blob/master/FUNet_Example_Use.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# **1. We import**

In [1]:
! git clone https://github.com/lgy112112/FUNet.git

Cloning into 'FUNet'...
remote: Enumerating objects: 19, done.
remote: Counting objects: 100% (19/19), done.
remote: Compressing objects: 100% (19/19), done.
remote: Total 19 (delta 3), reused 11 (delta 0), pack-reused 0
Receiving objects: 100% (19/19), 8.34 KiB | 8.34 MiB/s, done.
Resolving deltas: 100% (3/3), done.


# **2. We get**

In [3]:
from FUNet.unet_code_generator import *

# **3. We print and copy**

In [4]:
code_text = unet_code_generator(depth=5, n_channels=3, n_classes=10, bilinear=False)
print(code_text)

from FUNet.unet_parts import *

class UNet(nn.Module):
    def __init__(self, n_channels=3, n_classes=10, bilinear=False):
        super(UNet, self).__init__()
        self.n_channels = n_channels
        self.n_classes = n_classes
        self.bilinear = bilinear
        factor = 2 if bilinear else 1

        self.inc = DoubleConv(n_channels, 64)
        self.down1 = Down(64, 128)
        self.down2 = Down(128, 256)
        self.down3 = Down(256, 512)
        self.down4 = Down(512, 1024 // factor)

        self.up4 = Up(1024, 512 // factor, bilinear)
        self.up3 = Up(512, 256 // factor, bilinear)
        self.up2 = Up(256, 128 // factor, bilinear)
        self.up1 = Up(128, 64, bilinear)
        self.outc = OutConv(64, n_classes)

    def forward(self, x):
        x1 = self.inc(x)
        x2 = self.down1(x1)
        x3 = self.down2(x2)
        x4 = self.down3(x3)
        x5 = self.down4(x4)
        x = self.up4(x5, x4)
        x = self.up3(x, x3)
        x = self.up2(x, x2)
     

# **4. We paste**

In [5]:
from FUNet.unet_parts import *

class UNet(nn.Module):
    def __init__(self, n_channels=3, n_classes=10, bilinear=False):
        super(UNet, self).__init__()
        self.n_channels = n_channels
        self.n_classes = n_classes
        self.bilinear = bilinear
        factor = 2 if bilinear else 1

        self.inc = DoubleConv(n_channels, 64)
        self.down1 = Down(64, 128)
        self.down2 = Down(128, 256)
        self.down3 = Down(256, 512)
        self.down4 = Down(512, 1024 // factor)

        self.up4 = Up(1024, 512 // factor, bilinear)
        self.up3 = Up(512, 256 // factor, bilinear)
        self.up2 = Up(256, 128 // factor, bilinear)
        self.up1 = Up(128, 64, bilinear)
        self.outc = OutConv(64, n_classes)

    def forward(self, x):
        x1 = self.inc(x)
        x2 = self.down1(x1)
        x3 = self.down2(x2)
        x4 = self.down3(x3)
        x5 = self.down4(x4)
        x = self.up4(x5, x4)
        x = self.up3(x, x3)
        x = self.up2(x, x2)
        x = self.up1(x, x1)
        logits = self.outc(x)
        return logits

# **5. We drive**<img src="https://i.kym-cdn.com/entries/icons/original/000/047/390/cover8.jpg" alt="My Image">

In [6]:
model = UNet(n_channels=3, n_classes=10, bilinear=False)
test_input = torch.randn(1, 3, 256, 256)
output = model(test_input)
print(output.shape)

torch.Size([1, 10, 256, 256])
